In [4]:
from keras.datasets import cifar100

(X_train, y_train), (X_test, y_test) = cifar100.load_data()

169009152/169001437 [==============================] - 6s 0us/step


In [5]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

y_train.shape, y_test.shape

((50000, 100), (10000, 100))

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=2045)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((35000, 32, 32, 3), (35000, 100), (10000, 32, 32, 3), (10000, 100))

In [7]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet', include_top = False, input_shape=(32,32,3))
conv_base.summary()

58892288/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
______________________________________________________________

In [9]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

datagen = ImageDataGenerator(rescale= 1./255)
batch_size = 32

def extract_features(x, y, sample_count):
  features = np.zeros(shape = (sample_count, 1, 1, 512))
  labels = np.zeros(shape = (sample_count, 100))

  generator = datagen.flow(x, y)

  i = 0
  for input_batch, label_batch in generator:
    features_batch = conv_base.predict(input_batch)
    features[i*batch_size : (1+i) * batch_size ] = features_batch
    labels[i*batch_size : (i+1) * batch_size] = label_batch
    i+=1
    if i*batch_size >=sample_count:
      break
    return features, labels
  
X_train, y_train = extract_features(X_train, y_train, 35000)
X_valid, y_valid = extract_features(X_valid, y_valid, 15000)
X_test, y_test = extract_features(X_test, y_test, 10000)

print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape, y_test.shape)

(35000, 1, 1, 512) (35000, 100) (15000, 1, 1, 512) (15000, 100) (10000, 1, 1, 512) (10000, 100)


In [10]:
X_train = np.reshape(X_train, (35000,512))
X_valid = np.reshape(X_valid, (15000, 512))
X_test = np.reshape(X_test, (10000, 512))

X_train.shape, X_valid.shape, X_test.shape

((35000, 512), (15000, 512), (10000, 512))

In [11]:
from keras import models, layers

model = models.Sequential()
model.add(layers.Dense(1024, activation='relu', input_dim=(512)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(100, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               102500    
Total params: 627,812
Trainable params: 627,812
Non-trainable params: 0
_________________________________________________________________


In [ ]:
%%time

Hist = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_valid, y_valid))